# Tensors
[Torch docs on tensors](https://pytorch.org/docs/stable/tensors.html) 

In [31]:
import torch
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

torch.__version__

Using device: cpu


'2.0.1+cpu'

## Creating Tensors  
From a python list

In [32]:
torch.tensor([[2 , 3],[4 , 5]])

tensor([[2, 3],
        [4, 5]])

From a numpy array

In [33]:
v = np.array([[[1,2],[2,3],[4,5]]])
t = torch.tensor(v)

From random values

In [34]:
torch.rand(size=(1,3,3))

tensor([[[0.0716, 0.0323, 0.7047],
         [0.2545, 0.3994, 0.2122],
         [0.4089, 0.1481, 0.1733]]])

In a range of values

In [35]:
torch.arange(0,10,1)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

[Torch docs on how to create tensors](https://pytorch.org/docs/stable/torch.html#tensor-creation-ops)

## Tensor attributes
[Docs on tensor attributes](https://pytorch.org/docs/stable/tensor_attributes.html#tensor-attributes-doc)

In [36]:
v = np.array([[[1,2],[2,3],[4,5]]])
t = torch.tensor(v)

In [37]:
print(f"Number of dimensions is {t.ndim}")
print(f"Shape is {t.shape}") #1 dimension of 3x2 matrices (always from out to in and row first)
print(t.device)
print(f"Datatype is {t.dtype}") #float64 is default in numpy
t = t.type(torch.float32) #changing datatype of the tensor into float32
print(t) #the datatype is not printed because float32 is default in pytorch
print(f"Number of elements is {t.numel()}")

Number of dimensions is 3
Shape is torch.Size([1, 3, 2])
cpu
Datatype is torch.int64
tensor([[[1., 2.],
         [2., 3.],
         [4., 5.]]])
Number of elements is 6


## Tensor manipulation

### Applying a linear transformation to the incoming data:  $ y = x A^T + b $

In [1]:
import torch
x = torch.tensor([[1, 2],
                  [3, 4],
                  [5, 6]], dtype=torch.float32)
# Since the linear layer starts with a random weights matrix A, let's make it reproducible (more on this later)
torch.manual_seed(42)
# This uses matrix multiplication
x = x.T
linear = torch.nn.Linear(in_features=3, # in_features = matches inner dimension of input (second index of x, aka its columns)
                         out_features=4, # out_features = describes outer value (second index of A, aka its columns)
                         bias = True)
output = linear(x)
print(f"Input shape: {x.shape}\n")
print(f"Output:{output}\nOutput shape: {output.shape}")

Input shape: torch.Size([2, 3])

Output:tensor([[1.6293, 0.8116, 3.5593, 1.5407],
        [2.4146, 1.3319, 4.1262, 1.7270]], grad_fn=<AddmmBackward0>)
Output shape: torch.Size([2, 4])


### Aggregating tensors

In [39]:
v = torch.arange(0, 100, 10)
print(v)
print(f"Maximum {torch.max(v)} at index {torch.argmax(v)}")
print(f"Minimum {torch.min(v)} at index {torch.argmin(v)}")
print(f"Mean: {torch.mean(v.type(torch.float32))}")
print(f"Sum: {torch.sum(v)}")

tensor([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])
Maximum 90 at index 9
Minimum 0 at index 0
Mean: 45.0
Sum: 450


### Indexing and slicing

In [40]:
torch.manual_seed(42)
v = torch.rand(2,3,3)
v
v[-2] #negative indices go from last to first element
#putting only one index means taking only the first dimension
v[0,0,0]

tensor(0.8823)

In [41]:
v[1:3] # is the same as line below

tensor([[[0.1332, 0.9346, 0.5936],
         [0.8694, 0.5677, 0.7411],
         [0.4294, 0.8854, 0.5739]]])

In [42]:
v[1:3,:,:]

tensor([[[0.1332, 0.9346, 0.5936],
         [0.8694, 0.5677, 0.7411],
         [0.4294, 0.8854, 0.5739]]])

In [43]:
v[0:3,2] = torch.tensor([1,2,3])
v

tensor([[[0.8823, 0.9150, 0.3829],
         [0.9593, 0.3904, 0.6009],
         [1.0000, 2.0000, 3.0000]],

        [[0.1332, 0.9346, 0.5936],
         [0.8694, 0.5677, 0.7411],
         [1.0000, 2.0000, 3.0000]]])

### Changing the shape of tensors

In [44]:
v = torch.arange(0,12,1,dtype=torch.float32)
v.shape

torch.Size([12])

#### Squeeze and unsqueeze

In [45]:
v1 = v.unsqueeze(dim=0) #adds a "dummy" (=1) dimension at dim index
print(v1)
v1.shape

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.]])


torch.Size([1, 12])

In [46]:
v1 = v1.squeeze(dim=0) #removes a "dummy" (=1) dimension at dim index
print(v1)
v1.shape

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])


torch.Size([12])

#### Reshape

In [47]:
#reshape can't change numel, which means that the product of dimensions must be invariant
#for that reason, one dimension can be -1 because python will do the rest 
v1 = v.reshape([2,1,-1])
v1,v1.shape

(tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.]],
 
         [[ 6.,  7.,  8.,  9., 10., 11.]]]),
 torch.Size([2, 1, 6]))

#### Stack
- Puts on top of each other the tensors, creating a new dimension in position dim which will correspond to the number of stacked tensors
- The other dimensions are the ones already present in the tensor

In [53]:
stacked = torch.stack([v,v],dim=0)
print(stacked)
stacked.shape

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.]])


torch.Size([2, 12])

#### Cat
- Concatenates tensor along the same dimension, without creating a new one (like stack)
- If in the first example below we would choose dim = 1, an error would rise (v has just one dimension)

In [54]:
concatenated = torch.cat([v,v],dim=0)
print(concatenated)
concatenated.shape

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,  0.,  1.,
         2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])


torch.Size([24])

In [59]:
v1 = torch.rand((2,1))
print(v1)
v2 = torch.rand((2,1))
print(v2)
torch.cat((v1,v2),dim = 1)
torch.cat((torch.zeros(1,1),v1),dim=0)

tensor([[0.7886],
        [0.5895]])
tensor([[0.7539],
        [0.1952]])


tensor([[0.0000],
        [0.7886],
        [0.5895]])

#### Broadcasting

When operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing (i.e. rightmost) dimension and works its way left. Two dimensions are compatible when
- they are equal
- one of them is 1

In [50]:
x = torch.randn((3, 2), requires_grad=True)
print(f"x = {x} \n")
y = x.reshape((1, 6))
z = torch.ones((4, 1))
print(z)
(y + z)

x = tensor([[ 0.3930,  0.4327],
        [-1.3627,  1.3564],
        [ 0.6688, -0.7077]], requires_grad=True) 

tensor([[1.],
        [1.],
        [1.],
        [1.]])


tensor([[ 1.3930,  1.4327, -0.3627,  2.3564,  1.6688,  0.2923],
        [ 1.3930,  1.4327, -0.3627,  2.3564,  1.6688,  0.2923],
        [ 1.3930,  1.4327, -0.3627,  2.3564,  1.6688,  0.2923],
        [ 1.3930,  1.4327, -0.3627,  2.3564,  1.6688,  0.2923]],
       grad_fn=<AddBackward0>)